In [45]:
import numpy as np

import json, time, faiss, numpy as np
from pathlib import Path
from typing import List, Dict
from sentence_transformers import SentenceTransformer

In [46]:
np.random.seed(42)  # reproducibility - neccessary

# problem list placeholder
problems = []
for i in range(100000):
    difficulty = ['easy', 'medium', 'hard'][i % 3]
    base_tags = ['array', 'string', 'dp', 'graph', 'math', 'greedy', 'binary search']
    # Pick 2 consecutive tags
    start_idx = i % len(base_tags)
    tags = base_tags[start_idx:start_idx+2]
    if len(tags) < 2:
        tags = base_tags[-2:]  # fallback

    # Random embedding (384-dim, like all-MiniLM-L6-v2)
    embedding = np.random.rand(384).astype('float32')

    problems.append({
        'id': i,
        'difficulty': difficulty,
        'tags': tags,
        'embedding': embedding
    })

print(f"✅ Generated {len(problems)} mock problems.")
print("Sample:", problems[0])

✅ Generated 100000 mock problems.
Sample: {'id': 0, 'difficulty': 'easy', 'tags': ['array', 'string'], 'embedding': array([0.37454012, 0.9507143 , 0.7319939 , 0.5986585 , 0.15601864,
       0.15599452, 0.05808361, 0.8661761 , 0.601115  , 0.7080726 ,
       0.02058449, 0.96990985, 0.83244264, 0.21233912, 0.18182497,
       0.1834045 , 0.30424225, 0.52475643, 0.43194503, 0.29122913,
       0.6118529 , 0.13949387, 0.29214466, 0.36636186, 0.45606998,
       0.785176  , 0.19967379, 0.5142344 , 0.59241456, 0.04645041,
       0.60754484, 0.17052412, 0.06505159, 0.94888556, 0.965632  ,
       0.80839735, 0.30461377, 0.09767211, 0.684233  , 0.4401525 ,
       0.12203824, 0.4951769 , 0.03438852, 0.9093204 , 0.25877997,
       0.66252226, 0.31171107, 0.52006805, 0.54671025, 0.18485446,
       0.96958464, 0.77513283, 0.93949896, 0.89482737, 0.5979    ,
       0.9218742 , 0.08849251, 0.19598286, 0.04522729, 0.32533032,
       0.3886773 , 0.27134904, 0.8287375 , 0.35675332, 0.2809345 ,
       0.5426

In [47]:
MODEL = SentenceTransformer("all-MiniLM-L6-v2")   # 384-dim, ~50 MB, 1 ms/query

class ProblemRecommender:
    """
    1. Load FAISS index once at start-up.
    2. Re-rank entire catalogue every time (but <150 ms).
    3. Thread-safe (read-only after init).
    """
    def __init__(self,
                 faiss_index_path: str,
                 problems_json_path: str,
                 top_k: int = 10):
        self.index = faiss.read_index(faiss_index_path)
        self.problems: List[Dict] = json.loads(Path(problems_json_path).read_text())
        self.top_k = top_k
        self.diff_buckets = {"easy": [], "medium": [], "hard": []}
        for idx, p in enumerate(self.problems):
            self.diff_buckets[p["difficulty"]].append(idx)


In [48]:
embeddings = np.array([p['embedding'] for p in problems]).astype('float32')
faiss.normalize_L2(embeddings)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = cosine
index.add(embeddings)

faiss.write_index(index, "problembank.faiss")
print("FAISS index built and saved to 'problembank.faiss'")

FAISS index built and saved to 'problembank.faiss'


In [49]:
loaded_index = faiss.read_index("problembank.faiss")
print(f"Loaded FAISS index with {loaded_index.ntotal} vectors")

Loaded FAISS index with 100000 vectors


In [50]:
def get_similar_by_embedding(query_embedding: np.ndarray, faiss_index, top_k: int = 50) -> List[int]:
    """
    Search FAISS index for top_k most similar problems.
    Returns list of indices.
    """
    query = query_embedding.reshape(1, -1)

    # Search
    _, indices = faiss_index.search(query, top_k)
    return indices[0].tolist()

# testing
sample_embedding = problems[0]['embedding']
similar_ids = get_similar_by_embedding(sample_embedding, loaded_index, top_k=5)
print("Top 5 similar to problem 0:", similar_ids)

Top 5 similar to problem 0: [0, 19922, 20019, 36232, 97508]


In [51]:
def compute_tag_score(candidate_tags: List[str], reference_tags: List[str]) -> float:
    """
    Compute normalized tag overlap score.
    Example: candidate=['dp','graph'], reference=['dp','array','graph'] → 2/3 ≈ 0.67
    """
    if not reference_tags:
        return 0.0
    common_tags = set(candidate_tags) & set(reference_tags)
    return len(common_tags) / len(reference_tags)

# testing
score = compute_tag_score(['dp', 'graph'], ['dp', 'array', 'graph'])
print("Tag score:", score)

Tag score: 0.6666666666666666


In [52]:
def rerank_recommendations(
    last_solved_indices: List[int],
    difficulty_filter: str,
    problems: List[Dict],
    faiss_index,
    top_k: int = 20
) -> List[Dict]:
    """
    Rerank problems using:
      40%: embedding similarity to last 2 solved
      60%: tag overlap with last 5 solved
    Only recommend problems matching difficulty_filter.
    """
    start_time = time.time()

    # --- Part A: Get average embedding of last 2 solved ---
    if len(last_solved_indices) >= 2:
        last_2_indices = last_solved_indices[-2:]
    elif len(last_solved_indices) == 1:
        last_2_indices = last_solved_indices
    else:
        last_2_indices = []

    if last_2_indices:
        avg_embedding = np.mean(
            [problems[i]['embedding'] for i in last_2_indices],
            axis=0
        ).astype('float32')
        # Get 100 most similar by embedding
        candidate_indices = get_similar_by_embedding(avg_embedding, faiss_index, top_k=100)
    else:
        # No history → use first 100 problems as candidates
        candidate_indices = list(range(min(100, len(problems))))

    # --- Part B: Collect tags from last 5 solved ---
    last_5_indices = last_solved_indices[-5:] if len(last_solved_indices) >= 5 else last_solved_indices
    reference_tags = []
    for idx in last_5_indices:
        reference_tags.extend(problems[idx]['tags'])

    # --- Part C: Score each candidate ---
    scored_candidates = []

    for idx in candidate_indices:
        prob = problems[idx]
        if prob['difficulty'] != difficulty_filter:
            continue  # skip if difficulty doesn't match

        # Compute embedding similarity score (40%)
        if last_2_indices:
            candidate_emb = prob['embedding']
            cos_sim = np.dot(candidate_emb, avg_embedding) / (
                np.linalg.norm(candidate_emb) * np.linalg.norm(avg_embedding) + 1e-8
            )
            similarity_score = max(0.0, min(1.0, cos_sim))  # clamp [0,1]
        else:
            similarity_score = 0.5  # neutral default

        # Compute tag score (60%)
        tag_score = compute_tag_score(prob['tags'], reference_tags)

        # Combine
        final_score = 0.4 * similarity_score + 0.6 * tag_score

        scored_candidates.append({
            'problem_id': prob['id'],
            'index': idx,
            'score': final_score,
            'similarity_score': similarity_score,
            'tag_score': tag_score,
            'tags': prob['tags'],
            'difficulty': prob['difficulty']
        })

    # Sort by final score descending
    scored_candidates.sort(key=lambda x: x['score'], reverse=True)

    end_time = time.time()
    print(f"⏱️  Reranking took: {(end_time - start_time)*1000:.1f} ms")

    return scored_candidates[:top_k]

In [ ]:
# Simulate user history: indices of last 6 solved problems
last_solved = [10, 25, 33, 47, 88, 105]  # you can change these
difficulty = "medium"  # or 'easy', 'hard'

# Run the recommender
recommendations = rerank_recommendations(
    last_solved_indices=last_solved,
    difficulty_filter=difficulty,
    problems=problems,
    faiss_index=loaded_index,
    top_k=10
)

# Display results
print(f"\nTop {len(recommendations)} Recommendations for difficulty='{difficulty}':\n")
for i, rec in enumerate(recommendations, 1):
    print(f"{i:2d}. ID: {rec['problem_id']:3d} | "
          f"Score: {rec['score']:.4f} "
          f"(sim: {rec['similarity_score']:.3f}, tag: {rec['tag_score']:.3f}) | "
          f"Tags: {rec['tags']}")

⏱️  Reranking took: 30.8 ms

🎯 Top 10 Recommendations for difficulty='medium':

 1. ID:  88 | Score: 0.4925 (sim: 0.931, tag: 0.200) | Tags: ['math', 'greedy']
 2. ID: 60787 | Score: 0.4596 (sim: 0.849, tag: 0.200) | Tags: ['greedy', 'binary search']
 3. ID: 87049 | Score: 0.4595 (sim: 0.849, tag: 0.200) | Tags: ['math', 'greedy']
 4. ID: 57253 | Score: 0.4595 (sim: 0.849, tag: 0.200) | Tags: ['array', 'string']
 5. ID: 29188 | Score: 0.4590 (sim: 0.848, tag: 0.200) | Tags: ['greedy', 'binary search']
 6. ID: 42145 | Score: 0.4589 (sim: 0.847, tag: 0.200) | Tags: ['greedy', 'binary search']
 7. ID: 61612 | Score: 0.4588 (sim: 0.847, tag: 0.200) | Tags: ['greedy', 'binary search']
 8. ID: 59674 | Score: 0.4585 (sim: 0.846, tag: 0.200) | Tags: ['greedy', 'binary search']
 9. ID: 93784 | Score: 0.4584 (sim: 0.846, tag: 0.200) | Tags: ['greedy', 'binary search']
10. ID: 34669 | Score: 0.4583 (sim: 0.846, tag: 0.200) | Tags: ['greedy', 'binary search']


In [54]:
timings = []
for _ in range(100):
    start = time.time()
    _ = rerank_recommendations(last_solved, "medium", problems, loaded_index, top_k=10)
    timings.append((time.time() - start) * 1000)  # ms

print(f"⏱️  Average latency: {np.mean(timings):.1f} ms")
print(f"⏱️  P95 latency: {np.percentile(timings, 95):.1f} ms")
print(f"✅ Target: <150ms → {'PASSED' if np.percentile(timings, 95) < 150 else 'FAILED'}")

⏱️  Reranking took: 25.6 ms
⏱️  Reranking took: 23.4 ms
⏱️  Reranking took: 21.2 ms
⏱️  Reranking took: 27.7 ms
⏱️  Reranking took: 21.0 ms
⏱️  Reranking took: 10.7 ms
⏱️  Reranking took: 23.4 ms
⏱️  Reranking took: 29.4 ms
⏱️  Reranking took: 15.9 ms
⏱️  Reranking took: 24.3 ms
⏱️  Reranking took: 10.5 ms
⏱️  Reranking took: 15.9 ms
⏱️  Reranking took: 17.0 ms
⏱️  Reranking took: 15.0 ms
⏱️  Reranking took: 15.9 ms
⏱️  Reranking took: 15.9 ms
⏱️  Reranking took: 15.9 ms
⏱️  Reranking took: 15.3 ms
⏱️  Reranking took: 14.7 ms
⏱️  Reranking took: 7.8 ms
⏱️  Reranking took: 7.9 ms
⏱️  Reranking took: 15.8 ms
⏱️  Reranking took: 16.0 ms
⏱️  Reranking took: 15.8 ms
⏱️  Reranking took: 20.9 ms
⏱️  Reranking took: 28.2 ms
⏱️  Reranking took: 25.6 ms
⏱️  Reranking took: 20.7 ms
⏱️  Reranking took: 0.8 ms
⏱️  Reranking took: 27.5 ms
⏱️  Reranking took: 16.0 ms
⏱️  Reranking took: 13.4 ms
⏱️  Reranking took: 12.4 ms
⏱️  Reranking took: 10.6 ms
⏱️  Reranking took: 21.0 ms
⏱️  Reranking took: 10.

Reranking took: 23.9 ms